## Overview ##

PubChem is a site run by the NIH which hosts raw data associated with chemical experiments; here we analyze the data hosted at PubChem for assay 1030, which looks for inhibitors of the protein encoding gene ALDH1A1. You can access the page for this assay [here](https://pubchem.ncbi.nlm.nih.gov/bioassay/1030)

## Results ##

We use the SMILES string, a common representation for a molecule amongst chemists, to begin the featurization process. Because the length of this string varies, it is normalized in the form of a Morgan Fingerprint; these are then used to train various DNN-based classifiers in this notebook

In [1]:
# Exploratory data analysis and visualization

In [1]:
import pickle
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, rdMolDescriptors
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.model_selection import train_test_split
import sys

import warnings
warnings.filterwarnings('ignore')

global_random_state = 42

np.random.seed(global_random_state)


active_pct = 0.073125471
inactive_pct = 1 - active_pct


import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

fh = logging.FileHandler('log_dnn.txt')
fh.setLevel(logging.INFO)
fh.setFormatter(formatter)
logger.addHandler(fh)

ch = logging.StreamHandler(sys.stdout)
ch.setLevel(logging.INFO)
ch.setFormatter(formatter)
logger.addHandler(ch)

In [5]:
# and downloading the datatable
from imblearn.under_sampling import RandomUnderSampler

ba_df = pd.read_csv("AID_1030_datatable_all.csv")

# Load compound info
cs_df = pd.read_csv("AID_1030_compound_smiles.csv",sep='\t',header=0)

# Merge the two
full_df = ba_df.merge(cs_df,on='PUBCHEM_CID')

# Cleanup the compound ID column
full_df["PUBCHEM_CID"] = full_df["PUBCHEM_CID"].astype(int)

# Delete CID 3246048, which fails featurization
full_df = full_df[full_df["PUBCHEM_CID"] != 3246048]

# Delete all inconclusive results
# Delete CID 3246048, which fails featurization
full_df = full_df[full_df["PUBCHEM_ACTIVITY_OUTCOME"] != "Inconclusive"]

compound_ids = list()
smiles_list = list()
fingerprints = list()
activities = list()

#fingerprint_df = 

for index, row in full_df.iterrows() :
    cid = row["PUBCHEM_CID"]
    smiles_string = row["Smiles"]
    mol = Chem.MolFromSmiles(smiles_string)
    is_active = row["PUBCHEM_ACTIVITY_OUTCOME"] == "Active"
    if mol is None:
        logger.info("Molecule failed featurization")
        logger.info(index)
    else: 
        fingerprint = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol,2,nBits=2048,useChirality=False,
                                                                     useBondTypes=False,useFeatures=False)
        
        # From RDKit documentation
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fingerprint, arr)
        fingerprint = arr
        
        compound_ids.append(cid)
        smiles_list.append(smiles_string)
        fingerprints.append(fingerprint)
        activities.append(is_active)
    
    if index % 10000 == 0:
        logger.info("Processed index: {0}".format(index))

fingerprints = np.array(fingerprints)
activities = np.array(activities,dtype=int)
        
logger.info("Sampling")

rus = RandomUnderSampler(random_state=global_random_state)
X, y = rus.fit_sample(fingerprints, activities)

logger.info("Processed all, pickling")

#compound_ids_and_features = (compound_ids, smiles_list, fingerprints, activities)

# Pickle the data to save time in the future
with open('data.classification.undersampled.pickle', 'wb') as f:
    pickle.dump((X,y), f, pickle.HIGHEST_PROTOCOL)

2017-09-27 11:29:52,381 - INFO - Processed index: 10000
2017-09-27 11:29:57,808 - INFO - Processed index: 30000
2017-09-27 11:30:00,628 - INFO - Processed index: 40000
2017-09-27 11:30:03,485 - INFO - Processed index: 50000
2017-09-27 11:30:06,415 - INFO - Processed index: 60000
2017-09-27 11:30:12,158 - INFO - Processed index: 80000
2017-09-27 11:30:14,984 - INFO - Processed index: 90000
2017-09-27 11:30:17,902 - INFO - Processed index: 100000
2017-09-27 11:30:20,756 - INFO - Processed index: 110000
2017-09-27 11:30:23,727 - INFO - Processed index: 120000
2017-09-27 11:30:26,784 - INFO - Processed index: 130000
2017-09-27 11:30:29,899 - INFO - Processed index: 140000
2017-09-27 11:30:32,974 - INFO - Processed index: 150000
2017-09-27 11:30:39,052 - INFO - Processed index: 170000
2017-09-27 11:30:50,860 - INFO - Processed index: 210000
2017-09-27 11:30:54,383 - INFO - Processed index: 220000
2017-09-27 11:30:54,839 - INFO - Sampling
2017-09-27 11:30:54,938 - INFO - Compute classes stat

In [6]:
import keras
print(keras.backend.backend())

tensorflow


In [9]:
# What about a deep neural network?
# Sample code from: https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

from keras.models import Sequential
from keras.layers import Dense
from keras import metrics
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.utils.class_weight import compute_class_weight

import pickle

k_fold_splits = 2
global_random_state = 42

with open('data.classification.undersampled.pickle', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    (X, y) = pickle.load(f)

#class_weights = compute_class_weight('balanced', np.unique(y), y)

    
def create_model() :
    model = Sequential()
    model.add(Dense(2000, input_dim=2048, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
    return model

skf = StratifiedKFold(n_splits=k_fold_splits,shuffle=True,random_state=global_random_state)

roc_auc_avg = 0

for train_index, test_index in skf.split(X,y) :

    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]

    classifier = KerasClassifier(build_fn=create_model, epochs=10, batch_size=100, verbose=1)
    classifier.fit(X_train,y_train)
    y_pred = classifier.predict(X_test)
    logger.info(classification_report(y_test, y_pred))
    # What is the AUC-ROC score?
    auc = roc_auc_score(y_test, y_pred, average='macro', sample_weight=None)
    roc_auc_avg = roc_auc_avg + auc
    
roc_auc_avg = roc_auc_avg / k_fold_splits
logger.info("Average roc_auc score of {} folds is: {}".format(k_fold_splits, roc_auc_avg))


Epoch 1/10
16110/16110 [==============================] - 1s - loss: 0.6033 - acc: 0.6732     
Epoch 2/10
16110/16110 [==============================] - 0s - loss: 0.4373 - acc: 0.7946     
Epoch 3/10
16110/16110 [==============================] - 0s - loss: 0.2361 - acc: 0.9022     
Epoch 4/10
16110/16110 [==============================] - 0s - loss: 0.0959 - acc: 0.9666     
Epoch 5/10
16110/16110 [==============================] - 0s - loss: 0.0463 - acc: 0.9857     
Epoch 6/10
16110/16110 [==============================] - 0s - loss: 0.0261 - acc: 0.9928     
Epoch 7/10
16110/16110 [==============================] - 0s - loss: 0.0132 - acc: 0.9958     
Epoch 8/10
16110/16110 [==============================] - 0s - loss: 0.0086 - acc: 0.9976     
Epoch 9/10
16110/16110 [==============================] - 0s - loss: 0.0159 - acc: 0.9959     
Epoch 10/10
11000/16112 [===================>..........] - ETA: 0s2017-09-27 11:39:47,851 - INFO -              precision    recall  f1-score   s

### 

In [10]:
# Let's try oversampling

from imblearn.over_sampling import RandomOverSampler

ba_df = pd.read_csv("AID_1030_datatable_all.csv")

# Load compound info
cs_df = pd.read_csv("AID_1030_compound_smiles.csv",sep='\t',header=0)

# Merge the two
full_df = ba_df.merge(cs_df,on='PUBCHEM_CID')

# Cleanup the compound ID column
full_df["PUBCHEM_CID"] = full_df["PUBCHEM_CID"].astype(int)

# Delete CID 3246048, which fails featurization
full_df = full_df[full_df["PUBCHEM_CID"] != 3246048]

# Delete all inconclusive results
# Delete CID 3246048, which fails featurization
full_df = full_df[full_df["PUBCHEM_ACTIVITY_OUTCOME"] != "Inconclusive"]

compound_ids = list()
smiles_list = list()
fingerprints = list()
activities = list()

#fingerprint_df = 

for index, row in full_df.iterrows() :
    cid = row["PUBCHEM_CID"]
    smiles_string = row["Smiles"]
    mol = Chem.MolFromSmiles(smiles_string)
    is_active = row["PUBCHEM_ACTIVITY_OUTCOME"] == "Active"
    if mol is None:
        logger.info("Molecule failed featurization")
        logger.info(index)
    else: 
        fingerprint = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol,2,nBits=2048,useChirality=False,
                                                                     useBondTypes=False,useFeatures=False)
        
        # From RDKit documentation
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fingerprint, arr)
        fingerprint = arr
        
        compound_ids.append(cid)
        smiles_list.append(smiles_string)
        fingerprints.append(fingerprint)
        activities.append(is_active)
    
    if index % 10000 == 0:
        logger.info("Processed index: {0}".format(index))

fingerprints = np.array(fingerprints)
activities = np.array(activities,dtype=int)
        
logger.info("Sampling")

ros = RandomOverSampler(random_state=global_random_state)
X, y = ros.fit_sample(fingerprints, activities)

logger.info("Processed all, pickling")

# Pickle the data to save time in the future
with open('data.classification.oversampled.pickle', 'wb') as f:
    pickle.dump((X,y), f, pickle.HIGHEST_PROTOCOL)

2017-09-27 11:41:51,224 - INFO - Processed index: 10000
2017-09-27 11:41:56,663 - INFO - Processed index: 30000
2017-09-27 11:41:59,473 - INFO - Processed index: 40000
2017-09-27 11:42:02,307 - INFO - Processed index: 50000
2017-09-27 11:42:05,228 - INFO - Processed index: 60000
2017-09-27 11:42:10,961 - INFO - Processed index: 80000
2017-09-27 11:42:13,811 - INFO - Processed index: 90000
2017-09-27 11:42:16,753 - INFO - Processed index: 100000
2017-09-27 11:42:19,614 - INFO - Processed index: 110000
2017-09-27 11:42:22,614 - INFO - Processed index: 120000
2017-09-27 11:42:25,669 - INFO - Processed index: 130000
2017-09-27 11:42:28,722 - INFO - Processed index: 140000
2017-09-27 11:42:31,770 - INFO - Processed index: 150000
2017-09-27 11:42:37,820 - INFO - Processed index: 170000
2017-09-27 11:42:50,020 - INFO - Processed index: 210000
2017-09-27 11:42:53,539 - INFO - Processed index: 220000
2017-09-27 11:42:54,013 - INFO - Sampling
2017-09-27 11:42:54,116 - INFO - Compute classes stat

In [12]:
# What about a deep neural network?
# Sample code from: https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

from keras.models import Sequential
from keras.layers import Dense
from keras import metrics
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.utils.class_weight import compute_class_weight

import pickle

k_fold_splits = 2
global_random_state = 42

with open('data.classification.oversampled.pickle', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    (X, y) = pickle.load(f)

#class_weights = compute_class_weight('balanced', np.unique(y), y)

print("Number of samples is: {}".format(X))
    
def create_model() :
    model = Sequential()
    model.add(Dense(2000, input_dim=2048, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
    return model

skf = StratifiedKFold(n_splits=k_fold_splits,shuffle=True,random_state=global_random_state)

roc_auc_avg = 0

for train_index, test_index in skf.split(X,y) :

    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
    
    print("Number of training samples is: {}".format(X_train))
    print("Number of test samples is: {}".format(X_test))

    classifier = KerasClassifier(build_fn=create_model, epochs=10, batch_size=100, verbose=1)
    classifier.fit(X_train,y_train)
    y_pred = classifier.predict(X_test)
    logger.info(classification_report(y_test, y_pred))
    # What is the AUC-ROC score?
    auc = roc_auc_score(y_test, y_pred, average='macro', sample_weight=None)
    roc_auc_avg = roc_auc_avg + auc
    
roc_auc_avg = roc_auc_avg / k_fold_splits
logger.info("Average roc_auc score of {} folds is: {}".format(k_fold_splits, roc_auc_avg))


Epoch 1/50
148298/148298 [==============================] - 8s - loss: 0.3842 - acc: 0.8283     
Epoch 2/50
148298/148298 [==============================] - 7s - loss: 0.1360 - acc: 0.9512     
Epoch 3/50
148298/148298 [==============================] - 7s - loss: 0.0625 - acc: 0.9792     
Epoch 4/50
148298/148298 [==============================] - 7s - loss: 0.0413 - acc: 0.9870     
Epoch 5/50
148298/148298 [==============================] - 7s - loss: 0.0310 - acc: 0.9907     
Epoch 6/50
148298/148298 [==============================] - 7s - loss: 0.0262 - acc: 0.9921     
Epoch 7/50
148298/148298 [==============================] - 7s - loss: 0.0209 - acc: 0.9939     
Epoch 8/50
148298/148298 [==============================] - 7s - loss: 0.0180 - acc: 0.9947     
Epoch 9/50
148298/148298 [==============================] - 7s - loss: 0.0161 - acc: 0.9951     
Epoch 10/50
148298/148298 [==============================] - 7s - loss: 0.0133 - acc: 0.9961     
Epoch 11/50
148298/148298 [==

148300/148300 [==============================] - 7s - loss: 0.0053 - acc: 0.9984     
Epoch 32/50
148300/148300 [==============================] - 7s - loss: 0.0049 - acc: 0.9986     
Epoch 33/50
148300/148300 [==============================] - 8s - loss: 0.0060 - acc: 0.9984     
Epoch 34/50
148300/148300 [==============================] - 8s - loss: 0.0055 - acc: 0.9982     
Epoch 35/50
148300/148300 [==============================] - 8s - loss: 0.0044 - acc: 0.9986     
Epoch 36/50
148300/148300 [==============================] - 8s - loss: 0.0044 - acc: 0.9986     
Epoch 37/50
148300/148300 [==============================] - 8s - loss: 0.0057 - acc: 0.9985     
Epoch 38/50
148300/148300 [==============================] - 7s - loss: 0.0054 - acc: 0.9984     
Epoch 39/50
148300/148300 [==============================] - 7s - loss: 0.0044 - acc: 0.9986     
Epoch 40/50
148300/148300 [==============================] - 7s - loss: 0.0046 - acc: 0.9986     
Epoch 41/50
148300/148300 [=====

In [13]:
# What is the performance of a dummy classifier on the training set?

from sklearn.dummy import DummyClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.utils.class_weight import compute_class_weight

import pickle

k_fold_splits = 2
global_random_state = 42

with open('data.classification.oversampled.pickle', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    (X, y) = pickle.load(f)

print("Number of samples is: {}".format(len(X)))
    
skf = StratifiedKFold(n_splits=k_fold_splits,shuffle=True,random_state=global_random_state)

roc_auc_avg = 0

for train_index, test_index in skf.split(X,y) :

    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
    
    print("Number of training samples is: {}".format(len(X_train)))
    print("Number of test samples is: {}".format(len(X_test)))

    classifier = DummyClassifier(random_state=rglobal_random_statew)
    classifier.fit(X_train,y_train)
    y_pred = classifier.predict(X_test)
    logger.info(classification_report(y_test, y_pred))
    # What is the AUC-ROC score?
    auc = roc_auc_score(y_test, y_pred, average='macro', sample_weight=None)
    roc_auc_avg = roc_auc_avg + auc
    
roc_auc_avg = roc_auc_avg / k_fold_splits
logger.info("Average roc_auc score of {} folds is: {}".format(k_fold_splits, roc_auc_avg))


Number of samples is: [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
Number of training samples is: [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
Number of test samples is: [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
2017-09-27 20:10:04,674 - INFO -              precision    recall  f1-score   support

          0       0.50      0.50      0.50     74150
          1       0.50      0.50      0.50     74150

avg / total       0.50      0.50      0.50    148300

Number of training samples is: [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.